In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Cargar datos
factores_h_trimestral = pd.read_csv(r'C:\Users\lucas\High-Frequency_GDP_Forecasting\src\modelo\prueba1.csv', delimiter=',')
factores_h_trimestral['_date_'] = pd.PeriodIndex(factores_h_trimestral['_date_'], freq='Q')
factores_h_trimestral.set_index('_date_', inplace=True)
factores_h_trimestral['uno'] = 1

pib_prueba = pd.read_csv(r'C:\Users\lucas\Downloads\PIB_prueba.csv', delimiter=';', decimal=',')
pib_prueba['_date_'] = pd.PeriodIndex(pib_prueba['_date_'], freq='Q')
pib_prueba.set_index('_date_', inplace=True)
pib_prueba['uno'] = 1

# Calcular variaciones porcentuales interanuales
columnas_pcy = {col: f'PCY_{col}' for col in pib_prueba.columns if col not in ['uno']}
pib_prueba = pib_prueba.assign(**{new_col: pib_prueba[old_col].pct_change(periods=4) * 100 for old_col, new_col in columnas_pcy.items()})
pib_prueba = pib_prueba.loc['2007Q1':'2024Q3']

# Variables exógenas por ecuación
variables_exogenas = {
    "PCY_CPR": ["uno", "FCP1F_H", "FCP2F_H", "FCP3F_H", "FCP4F_H", "FCP5F_H", "FCP6F_H", "FCP7F_H"],
    "PCY_CPU": ["uno", "FCPU1F_H", "FCPU2F_H", "FCPU3F_H"],
    "PCY_INV": ["uno", "FINV1F_H", "FINV2F_H", "FINV3F_H", "FINV4F_H", "FINV5F_H"],
    "PCY_EXPOR": ["uno", "FEXP1F_H", "FEXP2F_H", "FEXP3F_H", "FEXP4F_H"],
    "PCY_IMP": ["uno", "FIMP1F_H", "FIMP2F_H", "FIMP3F_H", "FIMP4F_H"],
    "PCY_AGRICUL": ["uno", "FAGR1F_H", "FAGR2F_H", "FAGR3F_H", "FAGR4F_H", "FAGR5F_H"],
    "PCY_INDUSTRIA": ["uno", "FIND1F_H", "FIND2F_H", "FIND3F_H", "FIND4F_H", "FIND5F_H", "FIND6F_H"],
    "PCY_CONSTRU": ["uno", "FCON1F_H", "FCON2F_H", "FCON3F_H", "FCON4F_H", "FCON5F_H"],
    "PCY_SERVI": ["uno", "FSER1F_H", "FSER2F_H", "FSER3F_H", "FSER4F_H", "FSER5F_H", "FSER6F_H"],
    "PCY_IMPUESTOS": ["uno", "FIMPU1F_H", "FIMPU2F_H", "FIMPU3F_H", "FIMPU4F_H"]
}

# Ajustar modelos OLS
resultados_modelos = {}
resultados_estadisticos = {}
for columna, predictores in variables_exogenas.items():
    if columna in pib_prueba.columns:
        factores_disponibles = [var for var in predictores if var in factores_h_trimestral.columns]
        if factores_disponibles:
            y, X = pib_prueba[columna].align(factores_h_trimestral[factores_disponibles], join="inner", axis=0)
            modelo = sm.OLS(y, X).fit()
            resultados_modelos[columna] = modelo
            resultados_estadisticos[columna] = {"sigma": modelo.mse_resid, "ro": 1 - (sm.stats.durbin_watson(modelo.resid) / 2)}

# Cargar datos mensuales
x_prueba = pd.read_csv(r'C:\Users\lucas\High-Frequency_GDP_Forecasting\src\modelo\Factores_h_mensuales.csv', delimiter=',')
x_prueba.index = pd.date_range(start="1995-01-01", end="2026-12-01", freq="MS")
x_prueba = x_prueba.loc['2007-01-01':'2025-12-01']
x_prueba['uno'] = 1

# Proyección mensual de PIB
Ymens_dict = {}
t = 5
df_estadisticos = pd.DataFrame(resultados_estadisticos).T
for variable, stats in df_estadisticos.iterrows():
    sigma, ro = stats['sigma'], stats['ro']
    if ro > 0:
        rot = np.exp(-0.00302 + 0.45908 * np.log(ro) - 0.05179 * np.log(ro)**2 + 0.01148 * np.log(ro)**3)

    y_impu, x_impu = pib_prueba[variable], factores_h_trimestral[variables_exogenas[variable]]
    BETAG = np.linalg.inv(x_impu.T @ x_impu) @ (x_impu.T @ y_impu)
    ERRmens = y_impu - (x_impu @ BETAG)
    ERRMENSA = np.full((len(y_impu) + t, 1), ERRmens.iloc[-1])
    ERRMENSA[:len(y_impu)] = ERRmens.values.reshape(-1, 1)
    # Asegurar que ERRMENSA tenga el mismo tamaño que x_prueba
    ERRMENSA_corrected = np.repeat(ERRMENSA.flatten(), 3)[:x_prueba.shape[0]]

    # Aplicar la corrección antes de sumar
    Ymens_dict[variable] = (x_prueba[variables_exogenas[variable]] @ BETAG).values.flatten() + ERRMENSA_corrected


Ymens_final = pd.DataFrame(Ymens_dict, index=x_prueba.index).rename(columns=lambda x: x.replace("PCY_", ""))
Ymens_final['PIB_Demanda_mensual'] = (0.596 * Ymens_final['CPR'] + 0.166 * Ymens_final['CPU'] +
                                       0.267 * Ymens_final['INV'] + 0.286 * Ymens_final['EXPOR'] -
                                       0.315 * Ymens_final['IMP'])
Ymens_final['PIB_Oferta_mensual'] = (Ymens_final['AGRICUL'] * 0.0249973 + Ymens_final['INDUSTRIA'] * 0.1472338 +
                                      Ymens_final['CONSTRU'] * 0.0539303 + Ymens_final['SERVI'] * 0.6866292 +
                                      Ymens_final['IMPUESTOS'] * 0.0872094)
Ytrimes_final = Ymens_final.resample('QE').mean().to_period('Q').astype(str).rename(columns=lambda x: x.replace("_mensual", "_trimestral"))
Ytrimes_final


,CPR,CPU,INV,EXPOR,IMP,AGRICUL,INDUSTRIA,CONSTRU,SERVI,IMPUESTOS,PIB_Demanda_trimestral,PIB_Oferta_trimestral
2007Q1,3.8470774193986457,4.888613045542637,5.400211840395421,8.687603168424051,10.310826894572697,6.913214110162837,1.0456487929343385,2.0862556135903927,5.239434954869588,0.22089138393188557,3.7829685032861273,4.056091760014733
2007Q2,3.6035187080112134,5.3276172473354455,4.759131815851336,7.294139334288388,8.375239223329116,6.780597094651899,1.2466942831233727,1.5076762751228816,5.379292875905691,-0.37607030017600285,3.7506933021224804,4.095144289029586
2007Q3,3.276601705457715,5.705995686215144,3.6460883992337343,9.641733145189619,9.398357134515065,7.301609601255863,1.1070062053815726,1.2677819263156132,5.047288799868426,-0.1102411178190353,3.6706086851119046,3.8698829246304522
2007Q4,3.207931737469007,4.512527349305983,2.6739251825644494,5.161376181191297,5.068600406276547,9.163983914800681,0.643866508631497,-0.0541787901670198,4.8120077782700745,-0.30189379002798766,3.2544893391046283,3.5986889643599427
2008Q1,2.394835885465437,5.545721881452707,-0.3646717881857418,3.143531453615978,1.9068685999165833,0.41452688862600734,0.2595592390635581,0.08177934897379806,3.7563527102983,0.9642751080309343,2.5489310393734024,2.7163036409057155
...,...,...,...,...,...,...,...,...,...,...,...,...
2024Q4,3.423604730890601,5.25558163476725,0.8930879564680949,4.008420509389935,3.7759707507474385,6.471913676541536,4.701062074960837,0.9478269773966362,3.1494065395693167,0.12419267457328105,3.108326934559221,3.078357455691254
2025Q1,1.8223991140529225,5.007716157091385,-0.5604044031997583,2.849074691945678,3.1706838934366464,5.071601823862495,2.304399141363831,0.08891547721865638,3.10760240712213,0.40018404453995277,1.5838727138622961,2.639527398066663
2025Q2,0.2910135604761109,5.029072130270307,-0.5308742209334353,1.884620307975073,1.1374765225877381,5.257077914768183,1.2884977954563839,0.4599085064010735,3.3485009339840297,0.5480669477849237,1.0472229421451391,2.6929012913751245
2025Q3,0.5383062244580982,4.971195401679826,0.06116839070376797,2.5631030645824535,1.869483656399715,5.1717160157471005,1.2769526189973268,0.06458879421515373,3.9708092397649346,0.4729608805552134,1.306541031478455,3.088493022593122


In [2]:
Ymens_final.to_pickle("Ymens_final.pkl")